# Set up

In [3]:
import argparse
import sys
import json
import logging
import sys
from copy import deepcopy

from gboml import GbomlGraph
from gboml.output import write_csv
from gboml.compiler import parse_file
from gboml.compiler.classes import Parameter, Expression, Constraint, Identifier, Node

class MyGbomlGraph(GbomlGraph):
    def update_global_parameter(self, param: Parameter):
        self.global_parameters = [x if x.name != param.name else param for x in self.global_parameters]
    def get_node_by_name(self, name: str):
        for node in self.list_nodes:
            if node.name == name:
                return node
        return None
    def get_hyperedge_by_name(self, name: str):
        for h in self.list_hyperedges:
            if h.name == name:
                return h
        return None

def get_named_symbol_in_list(l, name):
    for h in l:
        if h.name == name:
            return h
    return None

def get_node_in(element, name: str):
    if isinstance(element, GbomlGraph):
        l = element.list_nodes
    elif isinstance(element, Node):
        l = element.nodes
    else:
        raise Exception(f"Unknown class {element.__class__}")
    return get_named_symbol_in_list(l, name)
    

def get_hyperedge_in(element, name: str):
    if isinstance(element, GbomlGraph):
        l = element.list_hyperedges
    elif isinstance(element, Node):
        l = element.hyperedges
    else:
        raise Exception(f"Unknown class {element.__class__}")
    return get_named_symbol_in_list(l, name)

def update_parameter(symb, param):
    symb.parameters = [x if x.name != param.name else param for x in symb.parameters]

    
def build_model(time_horizon,model_name):
    program = parse_file(model_name)
    model = MyGbomlGraph()
    model.add_nodes_in_model(*deepcopy(program.get_nodes()))
    model.add_hyperedges_in_model(*deepcopy(program.get_links()))
    model.add_global_parameters(deepcopy(program.get_global_parameters()))
    model.set_timehorizon(time_horizon)

    return model

def build_and_solve(model, outfile=None):
    """
    Builds and solves the given model.
    :param model:
    :param outfile: if defined, the solution is dump as a json file at this location
    :return: the solution dict
    """
    model.build_model(8)

    print("START")
    solution, objective, status, solver_info, cai, vai = model.solve_gurobi(details="gurobi_detail.txt", opt_file="gurobi.opt.txt")
    out = model.turn_solution_to_dictionary(solver_info, status, solution, objective, cai, vai)
    outlist = model.turn_solution_to_list(solution, constraints_info=cai)
    list_names = [x[0] for x in outlist]
    list_val = [x[1] for x in outlist]
    if outfile:
        json.dump(out, open(outfile,'w'), indent=4, sort_keys=True)
        write_csv(outfile[:-5] + '.csv', list_names,
          list_val, transpose=True)
    
    return out

if __name__ == '__main__':
    name_file = "3_clusters_Belgium.txt" 
    file_path = "C:/Users/yaita/OneDrive/Bureau/Master Thesis/JocelynMbenoun-Integrationoffshorebelgium-9f04c10/" # file path for the output
    TimeHorizon = 8760

# Base case

In [2]:
name_file = "3_clusters_Belgium.txt"
TimeHorizon = 8760
model = build_model(TimeHorizon,name_file)
names_case = 'base_case'
Inland = get_node_in(model, "INLAND")
H2_NL = get_node_in(Inland, "H2_INTERCONNECTION_NL")
h2_import_costs = [0.075,0.1,0.105,0.108,0.1125,0.125,0.1375,0.15,0.1625,0.175,0.1875,0.2,0.225,0.25,0.275,0.3]
for cost in h2_import_costs:
    update_parameter(H2_NL, Parameter("import_cost", Expression("literal", cost)))
    name_outfile = file_path + "3_clusters_Belgium_" + names_case + "_cost_h2_" + str(cost) + ".json"
    out = build_and_solve(model, outfile = name_outfile)


Check variables of node INLAND : --- 0.0010008811950683594 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.0 seconds ---
Check variables of node PCCC_SMR : --- 0.0 seconds ---
Check variables of node PCCC_OCGT : --- 0.0009808540344238281 seconds ---
Check variables of node PCCC_CCGT : --- 0.0 seconds ---
Check variables of node PCCC_CHP : --- 0.0 seconds ---
Check variables of node PCCC_WS : --- 0.0 seconds ---
Check variables of node PCCC_BM : --- 0.0070683956146240234 seconds ---
Check variables of node DAC : --- 0.0 seconds ---
Check variables of node H2_STORAGE : --- 0.0010182857513427734 seconds ---
Check variables of node SMR : --- 0.0 seconds ---
Check variables of node H2_INTERCONNECTION_NL : --- 0.0 seconds ---
Check variables of node H2_INTERCONNECTION_DE : --- 0.0 seconds ---
Check variables of node H2_INTERCONNECTION : --- 0.0 seconds ---
Check variables of

  Matrix range     [1e-05, 9e+03]
  Objective range  [1e-06, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-05, 9e+05]
Presolve removed 3214952 rows and 1481028 columns
Presolve time: 8.43s
Presolved: 1040033 rows, 729128 columns, 3947132 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Elapsed ordering time = 6s
Ordering time: 9.07s

Barrier statistics:
 Dense cols : 46
 AA' NZ     : 7.151e+06
 Factor NZ  : 3.562e+07 (roughly 1.0 GB of memory)
 Factor Ops : 5.512e+09 (less than 1 second per iteration)
 Threads    : 6

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.95275486e+09 -5.61852609e+11  1.34e+08 1.98e+00  1.76e+06    20s
   1   1.92622966e+09 -5.49167024e+11  1.31e+08 5.42e+01  1.72e+06    21s
   2   1.90457057e+09 -4.21004600e+11  1.29e+08 3.99e+01  1.69e+06    22s
   3   1.71416048e+09 -2.93427535e+11  1.08e+08 2.97e

 101   2.21792583e+04  2.19806640e+04  8.86e-07 1.14e-09  9.28e-05   131s
 102   2.21762486e+04  2.19899331e+04  8.55e-07 1.04e-09  8.71e-05   132s
 103   2.21643471e+04  2.19980697e+04  7.36e-07 9.55e-10  7.77e-05   133s
 104   2.21568870e+04  2.20024423e+04  6.61e-07 9.12e-10  7.22e-05   135s
 105   2.21531219e+04  2.20072168e+04  6.24e-07 8.67e-10  6.82e-05   136s
 106   2.21484230e+04  2.20164608e+04  5.76e-07 7.72e-10  6.17e-05   137s
 107   2.21445350e+04  2.20218796e+04  5.37e-07 7.15e-10  5.73e-05   139s
 108   2.21423113e+04  2.20245348e+04  5.15e-07 6.88e-10  5.51e-05   140s
 109   2.21388736e+04  2.20299106e+04  4.80e-07 6.35e-10  5.09e-05   141s
 110   2.21349050e+04  2.20336857e+04  4.41e-07 5.94e-10  4.73e-05   143s
 111   2.21304381e+04  2.20360792e+04  3.96e-07 5.69e-10  4.41e-05   144s
 112   2.21272704e+04  2.20397174e+04  3.62e-07 5.34e-10  4.09e-05   145s
 113   2.21249025e+04  2.20427844e+04  3.38e-07 5.01e-10  3.84e-05   146s
 114   2.21226897e+04  2.20453561e+04 

  100381 DPushes remaining with DInf 0.0000000e+00               285s
   96533 DPushes remaining with DInf 0.0000000e+00               291s
   92596 DPushes remaining with DInf 0.0000000e+00               295s
   89961 DPushes remaining with DInf 0.0000000e+00               300s
   76585 DPushes remaining with DInf 0.0000000e+00               305s
   48101 DPushes remaining with DInf 0.0000000e+00               310s
   27923 DPushes remaining with DInf 0.0000000e+00               315s
   22513 DPushes remaining with DInf 0.0000000e+00               321s
   19286 DPushes remaining with DInf 0.0000000e+00               326s
   17677 DPushes remaining with DInf 0.0000000e+00               330s
   14349 DPushes remaining with DInf 0.0000000e+00               335s
   12304 DPushes remaining with DInf 0.0000000e+00               340s
   11282 DPushes remaining with DInf 0.0000000e+00               346s
    8727 DPushes remaining with DInf 0.0000000e+00               351s
    5661 DPushes rem

START

Reading Gurobi options from file gurobi.opt

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-10875H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 4254985 rows, 2192638 columns and 9548974 nonzeros
Model fingerprint: 0x488ffb31
Coefficient statistics:
  Matrix range     [1e-05, 9e+03]
  Objective range  [1e-06, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-05, 9e+05]
Presolve removed 3214952 rows and 1481028 columns
Presolve time: 8.38s
Presolved: 1040033 rows, 729128 columns, 3947132 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Elapsed ordering time = 5s
Ordering time: 8.64s

Barrier statistics:
 Dense cols : 46
 AA' NZ     : 7.151e+06
 Factor NZ  : 3.562e+07 (roughly 1.0 GB of memory)
 Factor Ops : 5.512e+09 (less than 1 second per iteration)
 Threads    : 6


  96   2.22004106e+04  2.20830859e+04  4.68e-07 1.19e-09  5.48e-05   124s
  97   2.21974283e+04  2.20867897e+04  4.37e-07 1.12e-09  5.17e-05   126s
  98   2.21923590e+04  2.20953167e+04  3.84e-07 9.86e-10  4.54e-05   127s
  99   2.21896771e+04  2.21019448e+04  3.57e-07 8.69e-10  4.10e-05   128s
 100   2.21828669e+04  2.21044306e+04  2.88e-07 8.32e-10  3.67e-05   129s
 101   2.21807091e+04  2.21055721e+04  2.64e-07 8.11e-10  3.51e-05   130s
 102   2.21788418e+04  2.21132159e+04  2.45e-07 6.85e-10  3.07e-05   131s
 103   2.21753041e+04  2.21155594e+04  2.10e-07 6.49e-10  2.79e-05   132s
 104   2.21734103e+04  2.21208497e+04  1.90e-07 5.61e-10  2.46e-05   134s
 105   2.21722592e+04  2.21238607e+04  1.78e-07 5.09e-10  2.26e-05   135s
 106   2.21689654e+04  2.21262708e+04  1.44e-07 4.73e-10  2.00e-05   136s
 107   2.21664192e+04  2.21322899e+04  1.18e-07 3.71e-10  1.60e-05   137s
 108   2.21657200e+04  2.21343260e+04  1.11e-07 3.37e-10  1.47e-05   138s
 109   2.21652086e+04  2.21351101e+04 

   11897 DPushes remaining with DInf 0.0000000e+00               396s
    8443 DPushes remaining with DInf 0.0000000e+00               400s
    3998 DPushes remaining with DInf 0.0000000e+00               406s
    1443 DPushes remaining with DInf 0.0000000e+00               410s
       0 DPushes remaining with DInf 0.0000000e+00               415s

   71592 PPushes remaining with PInf 3.0520979e-02               415s
   53156 PPushes remaining with PInf 1.0074450e-03               431s
   46120 PPushes remaining with PInf 7.5716899e-04               444s
   41675 PPushes remaining with PInf 1.4758609e-02               452s
   37709 PPushes remaining with PInf 1.5032520e-02               461s
   34784 PPushes remaining with PInf 1.4547456e-02               467s
   31797 PPushes remaining with PInf 1.4486549e-02               474s
   30714 PPushes remaining with PInf 1.4491469e-02               477s
   26067 PPushes remaining with PInf 1.4670032e-02               484s
   25554 PPushes re

# Base case elec

In [4]:
name_file = "3_clusters_Belgium.txt"
TimeHorizon = 8760
model = build_model(TimeHorizon,name_file)
names_case = 'base_case_elec'
Inland = get_node_in(model, "INLAND")
H2_NL = get_node_in(Inland, "H2_INTERCONNECTION_NL")
h2_import_costs = [0.075,0.1,0.105,0.108,0.1125,0.125,0.1375,0.15,0.1625,0.175,0.1875,0.2,0.225,0.25,0.275,0.3]
for cost in h2_import_costs:
    update_parameter(H2_NL, Parameter("import_cost", Expression("literal", cost)))
    model.update_global_parameter(Parameter("case_elec", Expression("literal", 1)))
    name_outfile = file_path + "3_clusters_Belgium_" + names_case + "_cost_h2_" + str(cost) + ".json"
    out = build_and_solve(model, outfile = name_outfile)
    

Check variables of node INLAND : --- 0.0 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.001001596450805664 seconds ---
Check variables of node PCCC_SMR : --- 0.0 seconds ---
Check variables of node PCCC_OCGT : --- 0.0 seconds ---
Check variables of node PCCC_CCGT : --- 0.0 seconds ---
Check variables of node PCCC_CHP : --- 0.0 seconds ---
Check variables of node PCCC_WS : --- 0.0 seconds ---
Check variables of node PCCC_BM : --- 0.0 seconds ---
Check variables of node DAC : --- 0.0 seconds ---
Check variables of node H2_STORAGE : --- 0.0015387535095214844 seconds ---
Check variables of node SMR : --- 0.0 seconds ---
Check variables of node H2_INTERCONNECTION_NL : --- 0.0 seconds ---
Check variables of node H2_INTERCONNECTION_DE : --- 0.0 seconds ---
Check variables of node H2_INTERCONNECTION : --- 0.0010373592376708984 seconds ---
Check variables of node LINEPACK_NG :

Showing barrier log only...

Elapsed ordering time = 6s
Ordering time: 9.35s

Barrier statistics:
 Dense cols : 45
 AA' NZ     : 7.064e+06
 Factor NZ  : 3.340e+07 (roughly 1.0 GB of memory)
 Factor Ops : 4.866e+09 (less than 1 second per iteration)
 Threads    : 6

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.64598673e+09 -5.61560334e+11  1.82e+08 1.98e+00  2.27e+06    20s
   1   2.61139668e+09 -5.49374374e+11  1.78e+08 5.29e+01  2.21e+06    20s
   2   2.58329384e+09 -4.29361350e+11  1.75e+08 3.91e+01  2.24e+06    21s
   3   2.29189056e+09 -2.92695330e+11  1.44e+08 2.79e+01  1.74e+06    22s
   4   2.03000859e+09 -2.50718476e+11  1.23e+08 6.54e+00  1.48e+06    23s
   5   7.82359341e+08 -6.49890404e+10  3.09e+07 4.02e+00  4.28e+05    24s
   6   4.01340185e+08 -1.90546438e+10  2.30e+06 1.92e-01  3.78e+04    25s
   7   2.48463860e+08 -9.92678722e+09  9.05e+05 4.60e-02  1.48e+04    26s
   8   1.3185851

 106   2.21867371e+04  2.20188752e+04  1.20e-06 5.36e-13  8.05e-05   124s
 107   2.21782820e+04  2.20231615e+04  1.08e-06 7.96e-13  7.43e-05   125s
 108   2.21734174e+04  2.20266161e+04  1.02e-06 2.05e-13  7.04e-05   126s
 109   2.21686955e+04  2.20321847e+04  9.51e-07 1.31e-12  6.54e-05   127s
 110   2.21653401e+04  2.20390084e+04  9.05e-07 1.62e-12  6.05e-05   128s
 111   2.21598602e+04  2.20422722e+04  8.29e-07 1.10e-12  5.64e-05   130s
 112   2.21565126e+04  2.20462156e+04  7.83e-07 2.10e-12  5.29e-05   131s
 113   2.21518904e+04  2.20499585e+04  7.18e-07 2.27e-12  4.89e-05   131s
 114   2.21468226e+04  2.20543592e+04  6.48e-07 1.41e-12  4.43e-05   132s
 115   2.21440958e+04  2.20558275e+04  6.11e-07 2.90e-12  4.23e-05   133s
 116   2.21415428e+04  2.20579436e+04  5.75e-07 1.72e-12  4.01e-05   134s
 117   2.21392753e+04  2.20599041e+04  5.41e-07 2.45e-12  3.80e-05   135s
 118   2.21349345e+04  2.20620616e+04  4.79e-07 2.37e-12  3.49e-05   136s
 119   2.21326235e+04  2.20657245e+04 

  158871 DPushes remaining with DInf 0.0000000e+00               320s
  154910 DPushes remaining with DInf 0.0000000e+00               326s
  151160 DPushes remaining with DInf 0.0000000e+00               331s
  147213 DPushes remaining with DInf 0.0000000e+00               335s
  137074 DPushes remaining with DInf 0.0000000e+00               340s
  101032 DPushes remaining with DInf 0.0000000e+00               345s
   91002 DPushes remaining with DInf 0.0000000e+00               351s
   72875 DPushes remaining with DInf 0.0000000e+00               355s
   64112 DPushes remaining with DInf 0.0000000e+00               360s
   60863 DPushes remaining with DInf 0.0000000e+00               366s
   57841 DPushes remaining with DInf 0.0000000e+00               371s
   56056 DPushes remaining with DInf 0.0000000e+00               375s
   53815 DPushes remaining with DInf 0.0000000e+00               381s
   52784 DPushes remaining with DInf 0.0000000e+00               386s
   51584 DPushes rem

START

Reading Gurobi options from file gurobi.opt

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-10875H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 4254985 rows, 2192638 columns and 9548974 nonzeros
Model fingerprint: 0xa825dd42
Coefficient statistics:
  Matrix range     [1e-05, 9e+03]
  Objective range  [1e-06, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-05, 9e+05]
Presolve removed 3232474 rows and 1498551 columns
Presolve time: 7.93s
Presolved: 1022511 rows, 711605 columns, 3859541 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Elapsed ordering time = 6s
Ordering time: 9.19s

Barrier statistics:
 Dense cols : 45
 AA' NZ     : 7.064e+06
 Factor NZ  : 3.340e+07 (roughly 1.0 GB of memory)
 Factor Ops : 4.866e+09 (less than 1 second per iteration)
 Threads    : 6


  96   2.22018437e+04  2.21216968e+04  5.32e-08 7.79e-13  3.84e-05   115s
  97   2.21985433e+04  2.21255580e+04  4.88e-08 2.39e-12  3.50e-05   116s
  98   2.21949207e+04  2.21303036e+04  4.39e-08 2.79e-12  3.10e-05   117s
  99   2.21889541e+04  2.21318511e+04  3.58e-08 2.98e-12  2.74e-05   118s
 100   2.21873085e+04  2.21340759e+04  3.35e-08 2.56e-12  2.55e-05   119s
 101   2.21862277e+04  2.21361194e+04  3.21e-08 1.03e-12  2.40e-05   121s
 102   2.21844697e+04  2.21379753e+04  2.97e-08 8.77e-13  2.23e-05   122s
 103   2.21810631e+04  2.21395062e+04  2.50e-08 1.14e-12  1.99e-05   123s
 104   2.21797846e+04  2.21413458e+04  2.32e-08 9.00e-13  1.84e-05   124s
 105   2.21780932e+04  2.21435253e+04  2.10e-08 1.71e-12  1.66e-05   125s
 106   2.21773544e+04  2.21454756e+04  1.99e-08 2.07e-12  1.53e-05   126s
 107   2.21761903e+04  2.21468987e+04  1.84e-08 1.73e-12  1.40e-05   127s
 108   2.21754973e+04  2.21489920e+04  1.75e-08 1.46e-12  1.27e-05   128s
 109   2.21743706e+04  2.21499105e+04 

   42788 PPushes remaining with PInf 7.4670589e-04               463s
   40475 PPushes remaining with PInf 1.1405471e-03               470s
   38186 PPushes remaining with PInf 1.0605791e-03               475s
   36059 PPushes remaining with PInf 1.5068264e-03               482s
   34671 PPushes remaining with PInf 1.4962424e-03               486s
   30906 PPushes remaining with PInf 1.0955435e-03               491s
   28190 PPushes remaining with PInf 7.5925216e-03               496s
   17685 PPushes remaining with PInf 1.0585350e+00               507s
   15304 PPushes remaining with PInf 1.2648520e-03               511s
   12249 PPushes remaining with PInf 1.2648520e-03               516s
    9211 PPushes remaining with PInf 1.2648520e-03               521s
    6175 PPushes remaining with PInf 1.2648520e-03               526s
    3138 PPushes remaining with PInf 1.2648520e-03               531s
     594 PPushes remaining with PInf 8.8329792e-04               536s
       0 PPushes rem

# High Renew

In [3]:
name_file = "3_clusters_Belgium.txt"
TimeHorizon = 8760
model = build_model(TimeHorizon,name_file)

Offshore = get_node_in(model, "OFFSHORE")
Zeebrugge = get_node_in(model, "ZEEBRUGGE")
Inland = get_node_in(model, "INLAND")

PV = get_node_in(Inland,"PV")
update_parameter(PV, Parameter("max_capacity", Expression("literal", 100)))
WON = get_node_in(Inland,"WIND_ONSHORE")
update_parameter(WON, Parameter("max_capacity", Expression("literal", 20.5)))
WOFF = get_node_in(Offshore,"WIND_OFFSHORE")
update_parameter(WOFF, Parameter("max_capacity", Expression("literal", 9.3)))
CCGT = get_node_in(Inland, "CCGT")
OCGT = get_node_in(Inland, "OCGT")
h2_import_costs = [0.075,0.1,0.105,0.108,0.1125,0.125,0.1375,0.15,0.1625,0.175,0.1875,0.2,0.225,0.25,0.275,0.3]
for cost in h2_import_costs:
    H2_NL = get_node_in(Inland, "H2_INTERCONNECTION_NL")
    update_parameter(H2_NL, Parameter("import_cost", Expression("literal", cost)))
    name_outfile = file_path + "3_clusters_Belgium_High_renew_cost_h2_" + str(cost)+ ".json"
    out = build_and_solve(model, outfile = name_outfile)

Check variables of node INLAND : --- 0.0 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.0009984970092773438 seconds ---
Check variables of node PCCC_SMR : --- 0.0 seconds ---
Check variables of node PCCC_OCGT : --- 0.0010023117065429688 seconds ---
Check variables of node PCCC_CCGT : --- 0.0 seconds ---
Check variables of node PCCC_CHP : --- 0.0 seconds ---
Check variables of node PCCC_WS : --- 0.0 seconds ---
Check variables of node PCCC_BM : --- 0.0009980201721191406 seconds ---
Check variables of node DAC : --- 0.0 seconds ---
Check variables of node H2_STORAGE : --- 0.0009999275207519531 seconds ---
Check variables of node SMR : --- 0.0 seconds ---
Check variables of node H2_INTERCONNECTION_NL : --- 0.0 seconds ---
Check variables of node H2_INTERCONNECTION_DE : --- 0.0 seconds ---
Check variables of node H2_INTERCONNECTION : --- 0.0009987354278564453 seconds ---


  Objective range  [1e-06, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-05, 9e+05]
Presolve removed 3214952 rows and 1481028 columns
Presolve time: 7.93s
Presolved: 1040033 rows, 729128 columns, 3947132 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Elapsed ordering time = 6s
Ordering time: 9.92s

Barrier statistics:
 Dense cols : 46
 AA' NZ     : 7.151e+06
 Factor NZ  : 3.562e+07 (roughly 1.0 GB of memory)
 Factor Ops : 5.512e+09 (less than 1 second per iteration)
 Threads    : 6

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.95454240e+09 -5.62610400e+11  1.34e+08 1.98e+00  1.76e+06    20s
   1   1.92799883e+09 -5.49907581e+11  1.31e+08 5.42e+01  1.72e+06    21s
   2   1.90632504e+09 -4.21583349e+11  1.29e+08 3.99e+01  1.69e+06    22s
   3   1.71570113e+09 -2.93777914e+11  1.09e+08 2.97e+01  1.33e+06    23s
   4   1.5122

 102   2.11157933e+04  2.10022303e+04  1.46e-07 1.09e-09  5.31e-05   131s
 103   2.11137452e+04  2.10117161e+04  1.38e-07 9.63e-10  4.77e-05   132s
 104   2.11101712e+04  2.10147253e+04  1.22e-07 9.22e-10  4.46e-05   133s
 105   2.11067609e+04  2.10216448e+04  1.08e-07 8.26e-10  3.98e-05   134s
 106   2.11050697e+04  2.10234078e+04  1.00e-07 8.03e-10  3.82e-05   135s
 107   2.11017871e+04  2.10264632e+04  8.67e-08 7.64e-10  3.52e-05   136s
 108   2.11002182e+04  2.10317618e+04  7.99e-08 6.91e-10  3.20e-05   137s
 109   2.10986204e+04  2.10356052e+04  7.30e-08 6.35e-10  2.95e-05   139s
 110   2.10980651e+04  2.10376000e+04  7.06e-08 6.12e-10  2.83e-05   140s
 111   2.10964994e+04  2.10392197e+04  6.38e-08 5.91e-10  2.68e-05   141s
 112   2.10955086e+04  2.10428404e+04  5.94e-08 5.40e-10  2.46e-05   142s
 113   2.10943502e+04  2.10447680e+04  5.43e-08 5.11e-10  2.32e-05   143s
 114   2.10930942e+04  2.10486762e+04  4.90e-08 4.60e-10  2.08e-05   144s
 115   2.10924136e+04  2.10508460e+04 

START

Reading Gurobi options from file gurobi.opt

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-10875H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 4254985 rows, 2192638 columns and 9548974 nonzeros
Model fingerprint: 0x2c6371cb
Coefficient statistics:
  Matrix range     [1e-05, 9e+03]
  Objective range  [1e-06, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-05, 9e+05]
Presolve removed 3214952 rows and 1481028 columns
Presolve time: 8.23s
Presolved: 1040033 rows, 729128 columns, 3947132 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Elapsed ordering time = 5s
Ordering time: 9.85s

Barrier statistics:
 Dense cols : 46
 AA' NZ     : 7.151e+06
 Factor NZ  : 3.562e+07 (roughly 1.0 GB of memory)
 Factor Ops : 5.512e+09 (less than 1 second per iteration)
 Threads    : 6


  96   2.11740382e+04  2.10618235e+04  2.00e-07 1.34e-09  5.25e-05   129s
  97   2.11693411e+04  2.10678380e+04  1.80e-07 1.22e-09  4.75e-05   131s
  98   2.11658040e+04  2.10731444e+04  1.65e-07 1.11e-09  4.33e-05   132s
  99   2.11631915e+04  2.10776532e+04  1.54e-07 1.02e-09  4.00e-05   133s
 100   2.11605196e+04  2.10808447e+04  1.42e-07 9.56e-10  3.72e-05   134s
 101   2.11564790e+04  2.10844522e+04  1.24e-07 8.81e-10  3.37e-05   135s
 102   2.11531636e+04  2.10879036e+04  1.09e-07 8.13e-10  3.05e-05   136s
 103   2.11523691e+04  2.10894324e+04  1.06e-07 7.83e-10  2.94e-05   138s
 104   2.11510460e+04  2.10932091e+04  9.98e-08 7.08e-10  2.70e-05   139s
 105   2.11492932e+04  2.10948827e+04  9.20e-08 6.75e-10  2.54e-05   140s
 106   2.11478320e+04  2.10971193e+04  8.52e-08 6.32e-10  2.37e-05   141s
 107   2.11467882e+04  2.11010524e+04  8.03e-08 5.53e-10  2.14e-05   142s
 108   2.11452685e+04  2.11017649e+04  7.36e-08 5.41e-10  2.03e-05   143s
 109   2.11437911e+04  2.11075037e+04 

   43774 DPushes remaining with DInf 0.0000000e+00               366s
   41570 DPushes remaining with DInf 0.0000000e+00               371s
   39257 DPushes remaining with DInf 0.0000000e+00               376s
   37414 DPushes remaining with DInf 0.0000000e+00               381s
   35835 DPushes remaining with DInf 0.0000000e+00               387s
   34788 DPushes remaining with DInf 0.0000000e+00               391s
   33738 DPushes remaining with DInf 0.0000000e+00               395s
   31099 DPushes remaining with DInf 0.0000000e+00               402s
   29134 DPushes remaining with DInf 0.0000000e+00               407s
   28107 DPushes remaining with DInf 0.0000000e+00               411s
   17531 DPushes remaining with DInf 0.0000000e+00               415s
   14510 DPushes remaining with DInf 0.0000000e+00               420s
   12028 DPushes remaining with DInf 0.0000000e+00               427s
   11006 DPushes remaining with DInf 0.0000000e+00               432s
   10495 DPushes rem

# High Renew elec

In [5]:
name_file = "3_clusters_Belgium.txt"
TimeHorizon = 8760
model = build_model(TimeHorizon,name_file)

Offshore = get_node_in(model, "OFFSHORE")
Zeebrugge = get_node_in(model, "ZEEBRUGGE")
Inland = get_node_in(model, "INLAND")

PV = get_node_in(Inland,"PV")
update_parameter(PV, Parameter("max_capacity", Expression("literal", 100)))
WON = get_node_in(Inland,"WIND_ONSHORE")
update_parameter(WON, Parameter("max_capacity", Expression("literal", 20.5)))
WOFF = get_node_in(Offshore,"WIND_OFFSHORE")
update_parameter(WOFF, Parameter("max_capacity", Expression("literal", 9.3)))
CCGT = get_node_in(Inland, "CCGT")
OCGT = get_node_in(Inland, "OCGT")
h2_import_costs = [0.075,0.1,0.105,0.108,0.1125,0.125,0.1375,0.15,0.1625,0.175,0.1875,0.2,0.225,0.25,0.275,0.3]
model.update_global_parameter(Parameter("case_elec", Expression("literal", 1)))
for cost in h2_import_costs:
    H2_NL = get_node_in(Inland, "H2_INTERCONNECTION_NL")
    update_parameter(H2_NL, Parameter("import_cost", Expression("literal", cost)))
    name_outfile = file_path + "3_clusters_Belgium_High_renew_elec_cost_h2_" + str(cost)+ ".json"
    out = build_and_solve(model, outfile = name_outfile)

Check variables of node INLAND : --- 0.0 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.001001119613647461 seconds ---
Check variables of node PCCC_SMR : --- 0.0 seconds ---
Check variables of node PCCC_OCGT : --- 0.0 seconds ---
Check variables of node PCCC_CCGT : --- 0.0009992122650146484 seconds ---
Check variables of node PCCC_CHP : --- 0.0 seconds ---
Check variables of node PCCC_WS : --- 0.0 seconds ---
Check variables of node PCCC_BM : --- 0.0 seconds ---
Check variables of node DAC : --- 0.0010235309600830078 seconds ---
Check variables of node H2_STORAGE : --- 0.0 seconds ---
Check variables of node SMR : --- 0.001010894775390625 seconds ---
Check variables of node H2_INTERCONNECTION_NL : --- 0.0 seconds ---
Check variables of node H2_INTERCONNECTION_DE : --- 0.0 seconds ---
Check variables of node H2_INTERCONNECTION : --- 0.0 seconds ---
Check variables of n

  RHS range        [2e-05, 9e+05]
Presolve removed 3232474 rows and 1498551 columns
Presolve time: 8.62s
Presolved: 1022511 rows, 711605 columns, 3859541 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Elapsed ordering time = 6s
Ordering time: 9.69s

Barrier statistics:
 Dense cols : 45
 AA' NZ     : 7.064e+06
 Factor NZ  : 3.340e+07 (roughly 1.0 GB of memory)
 Factor Ops : 4.866e+09 (less than 1 second per iteration)
 Threads    : 6

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.64933859e+09 -5.62317931e+11  1.82e+08 1.98e+00  2.27e+06    21s
   1   2.61471228e+09 -5.50115608e+11  1.78e+08 5.29e+01  2.21e+06    22s
   2   2.58658037e+09 -4.29932432e+11  1.75e+08 3.91e+01  2.25e+06    23s
   3   2.29471952e+09 -2.93062137e+11  1.45e+08 2.79e+01  1.74e+06    24s
   4   2.03254362e+09 -2.51025589e+11  1.23e+08 6.55e+00  1.48e+06    25s
   5   

 103   2.11794257e+04  2.11408361e+04  3.76e-08 1.17e-12  1.85e-05   136s
 104   2.11784923e+04  2.11413978e+04  3.54e-08 1.53e-12  1.78e-05   136s
 105   2.11771209e+04  2.11442734e+04  3.23e-08 2.84e-12  1.57e-05   138s
 106   2.11758858e+04  2.11453217e+04  2.95e-08 4.55e-12  1.46e-05   139s
 107   2.11751334e+04  2.11461520e+04  2.78e-08 1.14e-12  1.39e-05   140s
 108   2.11742772e+04  2.11480589e+04  2.59e-08 1.39e-12  1.26e-05   141s
 109   2.11735028e+04  2.11494701e+04  2.41e-08 7.11e-13  1.15e-05   142s
 110   2.11719021e+04  2.11499909e+04  2.05e-08 1.18e-12  1.05e-05   144s
 111   2.11714244e+04  2.11517039e+04  1.94e-08 2.72e-12  9.45e-06   145s
 112   2.11705200e+04  2.11523243e+04  1.74e-08 2.42e-12  8.72e-06   146s
 113   2.11697726e+04  2.11539139e+04  1.57e-08 1.27e-12  7.60e-06   147s
 114   2.11692201e+04  2.11541989e+04  1.45e-08 1.05e-12  7.20e-06   148s
 115   2.11686412e+04  2.11548076e+04  1.32e-08 1.80e-12  6.63e-06   149s
 116   2.11681178e+04  2.11554939e+04 

    8153 DPushes remaining with DInf 0.0000000e+00               366s
    6632 DPushes remaining with DInf 0.0000000e+00               371s
    5143 DPushes remaining with DInf 0.0000000e+00               376s
    3622 DPushes remaining with DInf 0.0000000e+00               382s
    2608 DPushes remaining with DInf 0.0000000e+00               386s
       0 DPushes remaining with DInf 0.0000000e+00               392s

   67143 PPushes remaining with PInf 2.4753204e-03               392s
   44965 PPushes remaining with PInf 1.0278627e-01               397s
   36730 PPushes remaining with PInf 1.0341999e-01               400s
   29050 PPushes remaining with PInf 1.4768139e-01               406s
   23316 PPushes remaining with PInf 1.0139674e-01               411s
   17485 PPushes remaining with PInf 9.2899453e-01               416s
   11376 PPushes remaining with PInf 3.2689629e-01               421s
    7328 PPushes remaining with PInf 3.2689629e-01               425s
    1757 PPushes re

START

Reading Gurobi options from file gurobi.opt

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-10875H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 4254985 rows, 2192638 columns and 9548974 nonzeros
Model fingerprint: 0x551b803e
Coefficient statistics:
  Matrix range     [1e-05, 9e+03]
  Objective range  [1e-06, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-05, 9e+05]
Presolve removed 3232474 rows and 1498551 columns
Presolve time: 8.50s
Presolved: 1022511 rows, 711605 columns, 3859541 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Elapsed ordering time = 8s
Ordering time: 12.46s

Barrier statistics:
 Dense cols : 45
 AA' NZ     : 7.064e+06
 Factor NZ  : 3.340e+07 (roughly 1.0 GB of memory)
 Factor Ops : 4.866e+09 (less than 1 second per iteration)
 Threads    : 6

  96   2.12216215e+04  2.11973164e+04  3.89e-08 1.25e-12  1.16e-05   139s
  97   2.12207337e+04  2.11990017e+04  3.64e-08 1.15e-12  1.04e-05   141s
  98   2.12187853e+04  2.11994945e+04  3.11e-08 8.80e-13  9.25e-06   142s
  99   2.12180161e+04  2.12000057e+04  2.90e-08 1.67e-12  8.63e-06   144s
 100   2.12171241e+04  2.12005422e+04  2.66e-08 1.49e-12  7.95e-06   145s
 101   2.12165617e+04  2.12009986e+04  2.50e-08 6.35e-13  7.46e-06   146s
 102   2.12160703e+04  2.12014567e+04  2.37e-08 6.18e-13  7.00e-06   148s
 103   2.12155248e+04  2.12020347e+04  2.21e-08 8.40e-13  6.47e-06   149s
 104   2.12150457e+04  2.12024138e+04  2.08e-08 6.54e-13  6.05e-06   150s
 105   2.12146444e+04  2.12026647e+04  1.96e-08 2.47e-12  5.74e-06   151s
 106   2.12138220e+04  2.12029865e+04  1.73e-08 1.76e-12  5.19e-06   153s
 107   2.12136127e+04  2.12033222e+04  1.67e-08 1.32e-12  4.93e-06   154s
 108   2.12130752e+04  2.12036129e+04  1.52e-08 1.78e-12  4.53e-06   155s
 109   2.12127487e+04  2.12038631e+04 

   65366 DPushes remaining with DInf 0.0000000e+00               367s
   64338 DPushes remaining with DInf 0.0000000e+00               372s
   62884 DPushes remaining with DInf 0.0000000e+00               376s
   60574 DPushes remaining with DInf 0.0000000e+00               381s
   51405 DPushes remaining with DInf 0.0000000e+00               385s
   46604 DPushes remaining with DInf 0.0000000e+00               391s
   45058 DPushes remaining with DInf 0.0000000e+00               395s
   41449 DPushes remaining with DInf 0.0000000e+00               400s
   28366 DPushes remaining with DInf 0.0000000e+00               406s
   24870 DPushes remaining with DInf 0.0000000e+00               411s
   21608 DPushes remaining with DInf 0.0000000e+00               416s
   19866 DPushes remaining with DInf 0.0000000e+00               422s
   18419 DPushes remaining with DInf 0.0000000e+00               427s
   17400 DPushes remaining with DInf 0.0000000e+00               433s
   16884 DPushes rem

# Distance Off - Coa base case

In [3]:
name_file = "3_clusters_Belgium_DC.txt"
TimeHorizon = 8760
model = build_model(TimeHorizon,name_file)
lengths = [40,50,60,70,80,90,100,120,150,200,500,750,1000]
max_power = [1, 0.98, 0.95, 0.91, 0.87, 0.82, 0.74, 0.62, 0.1] +[0] * 4  # consider lost in capacity in HVAC lines for long distance cables
max_HVDC = [100] * 13
sensitivity = 'off_zb'
i=0
for length in lengths:
    model.update_global_parameter(Parameter("length_off_zb", Expression("literal", length)))
    model.update_global_parameter(Parameter("max_power", Expression("literal", max_power[i])))
    model.update_global_parameter(Parameter("max_HVDC", Expression("literal", max_HVDC[i])))
    name_outfile = file_path + "3_clusters_Belgium_DC_" + sensitivity + '_' +  str(length) + ".json"
    out = build_and_solve(model, outfile = name_outfile)
    print(name_outfile)
    i=+1

Check variables of node INLAND : --- 0.0 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.000997304916381836 seconds ---
Check variables of node PCCC_SMR : --- 0.0 seconds ---
Check variables of node PCCC_OCGT : --- 0.0 seconds ---
Check variables of node PCCC_CCGT : --- 0.002992391586303711 seconds ---
Check variables of node PCCC_CHP : --- 0.0 seconds ---
Check variables of node PCCC_WS : --- 0.0 seconds ---
Check variables of node PCCC_BM : --- 0.0009963512420654297 seconds ---
Check variables of node DAC : --- 0.0 seconds ---
Check variables of node H2_STORAGE : --- 0.0009982585906982422 seconds ---
Check variables of node SMR : --- 0.0 seconds ---
Check variables of node H2_INTERCONNECTION_NL : --- 0.0 seconds ---
Check variables of node H2_INTERCONNECTION_DE : --- 0.0010387897491455078 seconds ---
Check variables of node H2_INTERCONNECTION : --- 0.0 seconds ---
Ch

CPLEX Error  1262: No basis exists.
CPLEX Error  1264: No norms available.


Unable to retrieve  basis  information for variables
Unable to retrieve  dual_norms  information for variables
C:/Users/jocel/Documents/Doctorat/3_clusters/3_clusters_Belgium_DC_off_zb_40.json
Check variables of node INLAND : --- 0.0009975433349609375 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.000997304916381836 seconds ---
Check variables of node PCCC_SMR : --- 0.0 seconds ---
Check variables of node PCCC_OCGT : --- 0.0 seconds ---
Check variables of node PCCC_CCGT : --- 0.000997304916381836 seconds ---
Check variables of node PCCC_CHP : --- 0.0 seconds ---
Check variables of node PCCC_WS : --- 0.000997304916381836 seconds ---
Check variables of node PCCC_BM : --- 0.0 seconds ---
Check variables of node DAC : --- 0.0 seconds ---
Check variables of node H2_STORAGE : --- 0.000997304916381836 seconds ---
Check variables of node SMR : --- 0.0009975433349609375 second

CPLEX Error  1262: No basis exists.
CPLEX Error  1264: No norms available.


Unable to retrieve  basis  information for variables
Unable to retrieve  dual_norms  information for variables
C:/Users/jocel/Documents/Doctorat/3_clusters/3_clusters_Belgium_DC_off_zb_50.json
Check variables of node INLAND : --- 0.0009970664978027344 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.0009975433349609375 seconds ---
Check variables of node PCCC_SMR : --- 0.0 seconds ---
Check variables of node PCCC_OCGT : --- 0.0 seconds ---
Check variables of node PCCC_CCGT : --- 0.0009968280792236328 seconds ---
Check variables of node PCCC_CHP : --- 0.0 seconds ---
Check variables of node PCCC_WS : --- 0.000997781753540039 seconds ---
Check variables of node PCCC_BM : --- 0.0 seconds ---
Check variables of node DAC : --- 0.0 seconds ---
Check variables of node H2_STORAGE : --- 0.000997304916381836 seconds ---
Check variables of node SMR : --- 0.0009970664978027344 seco

...

Unable to retrieve  basis  information for variables
Unable to retrieve  dual_norms  information for variables
C:/Users/jocel/Documents/Doctorat/3_clusters/3_clusters_Belgium_DC_off_zb_500.json
Check variables of node INLAND : --- 0.0009975433349609375 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.0009965896606445312 seconds ---
Check variables of node PCCC_SMR : --- 0.0 seconds ---
Check variables of node PCCC_OCGT : --- 0.0 seconds ---
Check variables of node PCCC_CCGT : --- 0.0015375614166259766 seconds ---
Check variables of node PCCC_CHP : --- 0.0 seconds ---
Check variables of node PCCC_WS : --- 0.0 seconds ---
Check variables of node PCCC_BM : --- 0.0 seconds ---
Check variables of node DAC : --- 0.0010302066802978516 seconds ---
Check variables of node H2_STORAGE : --- 0.0010008811950683594 seconds ---
Check variables of node SMR : --- 0.0 seconds ---
Check v

CPLEX Error  1262: No basis exists.
CPLEX Error  1264: No norms available.


Unable to retrieve  basis  information for variables
Unable to retrieve  dual_norms  information for variables
C:/Users/jocel/Documents/Doctorat/3_clusters/3_clusters_Belgium_DC_off_zb_750.json
Check variables of node INLAND : --- 0.00099945068359375 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.0009999275207519531 seconds ---
Check variables of node PCCC_SMR : --- 0.0 seconds ---
Check variables of node PCCC_OCGT : --- 0.0 seconds ---
Check variables of node PCCC_CCGT : --- 0.0009944438934326172 seconds ---
Check variables of node PCCC_CHP : --- 0.0 seconds ---
Check variables of node PCCC_WS : --- 0.0 seconds ---
Check variables of node PCCC_BM : --- 0.0009963512420654297 seconds ---
Check variables of node DAC : --- 0.0 seconds ---
Check variables of node H2_STORAGE : --- 0.0009968280792236328 seconds ---
Check variables of node SMR : --- 0.0 seconds ---
Check var

CPLEX Error  1262: No basis exists.
CPLEX Error  1264: No norms available.


Unable to retrieve  basis  information for variables
Unable to retrieve  dual_norms  information for variables
C:/Users/jocel/Documents/Doctorat/3_clusters/3_clusters_Belgium_DC_off_zb_1000.json


# Distance Off - Coa base case elec

In [4]:
name_file = "3_clusters_Belgium_DC.txt"
TimeHorizon = 8760
model = build_model(TimeHorizon,name_file)
lengths = [40,50,60,70,80,90,100,120,150,200,500,750,1000]
max_power = [1, 0.98, 0.95, 0.91, 0.87, 0.82, 0.74, 0.62, 0.1] +[0] * 4  # consider lost in capacity in HVAC lines for long distance cables
max_HVDC = [100] * 13
sensitivity = 'elec_off_zb'
i=0
model.update_global_parameter(Parameter("case_elec", Expression("literal", 1)))
for length in lengths:
    model.update_global_parameter(Parameter("length_off_zb", Expression("literal", length)))
    model.update_global_parameter(Parameter("max_power", Expression("literal", max_power[i])))
    model.update_global_parameter(Parameter("max_HVDC", Expression("literal", max_HVDC[i])))
    name_outfile = file_path + "3_clusters_Belgium_DC_" + sensitivity + '_' +  str(length) + ".json"
    out = build_and_solve(model, outfile = name_outfile)
    print(name_outfile)
    i=+1

Check variables of node INLAND : --- 0.000997304916381836 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.0009975433349609375 seconds ---
Check variables of node PCCC_SMR : --- 0.0 seconds ---
Check variables of node PCCC_OCGT : --- 0.000997304916381836 seconds ---
Check variables of node PCCC_CCGT : --- 0.0 seconds ---
Check variables of node PCCC_CHP : --- 0.0 seconds ---
Check variables of node PCCC_WS : --- 0.000997304916381836 seconds ---
Check variables of node PCCC_BM : --- 0.0 seconds ---
Check variables of node DAC : --- 0.0 seconds ---
Check variables of node H2_STORAGE : --- 0.000997304916381836 seconds ---
Check variables of node SMR : --- 0.000997304916381836 seconds ---
Check variables of node H2_INTERCONNECTION_NL : --- 0.0 seconds ---
Check variables of node H2_INTERCONNECTION_DE : --- 0.0 seconds ---
Check variables of node H2_INTERCONNECTION : --- 0.0

CPLEX Error  1262: No basis exists.
CPLEX Error  1264: No norms available.


Unable to retrieve  basis  information for variables
Unable to retrieve  dual_norms  information for variables
C:/Users/jocel/Documents/Doctorat/3_clusters/3_clusters_Belgium_DC_elec_off_zb_40.json
Check variables of node INLAND : --- 0.0 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.000997304916381836 seconds ---
Check variables of node CO2_STORAGE : --- 0.0 seconds ---
Check variables of node PCCC_SMR : --- 0.0009975433349609375 seconds ---
Check variables of node PCCC_OCGT : --- 0.000997304916381836 seconds ---
Check variables of node PCCC_CCGT : --- 0.0 seconds ---
Check variables of node PCCC_CHP : --- 0.0 seconds ---
Check variables of node PCCC_WS : --- 0.0009970664978027344 seconds ---
Check variables of node PCCC_BM : --- 0.0 seconds ---
Check variables of node DAC : --- 0.0009975433349609375 seconds ---
Check variables of node H2_STORAGE : --- 0.0 seconds ---
Check variables of node SMR : --- 0.000997304916381836 

CPLEX Error  1262: No basis exists.
CPLEX Error  1264: No norms available.


Unable to retrieve  basis  information for variables
Unable to retrieve  dual_norms  information for variables
C:/Users/jocel/Documents/Doctorat/3_clusters/3_clusters_Belgium_DC_elec_off_zb_50.json
Check variables of node INLAND : --- 0.0009975433349609375 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.000997304916381836 seconds ---
Check variables of node PCCC_SMR : --- 0.000997304916381836 seconds ---
Check variables of node PCCC_OCGT : --- 0.0 seconds ---
Check variables of node PCCC_CCGT : --- 0.0 seconds ---
Check variables of node PCCC_CHP : --- 0.0009975433349609375 seconds ---
Check variables of node PCCC_WS : --- 0.0009970664978027344 seconds ---
Check variables of node PCCC_BM : --- 0.0 seconds ---
Check variables of node DAC : --- 0.0009975433349609375 seconds ---
Check variables of node H2_STORAGE : --- 0.0009970664978027344 seconds ---
Check variables of 

...

Unable to retrieve  basis  information for variables
Unable to retrieve  dual_norms  information for variables
C:/Users/jocel/Documents/Doctorat/3_clusters/3_clusters_Belgium_DC_elec_off_zb_500.json
Check variables of node INLAND : --- 0.0009975433349609375 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.0 seconds ---
Check variables of node PCCC_SMR : --- 0.0 seconds ---
Check variables of node PCCC_OCGT : --- 0.0 seconds ---
Check variables of node PCCC_CCGT : --- 0.0009975433349609375 seconds ---
Check variables of node PCCC_CHP : --- 0.0 seconds ---
Check variables of node PCCC_WS : --- 0.0 seconds ---
Check variables of node PCCC_BM : --- 0.0009970664978027344 seconds ---
Check variables of node DAC : --- 0.0 seconds ---
Check variables of node H2_STORAGE : --- 0.0009975433349609375 seconds ---
Check variables of node SMR : --- 0.0 seconds ---
Check variables of n

CPLEX Error  1262: No basis exists.
CPLEX Error  1264: No norms available.


Unable to retrieve  basis  information for variables
Unable to retrieve  dual_norms  information for variables
C:/Users/jocel/Documents/Doctorat/3_clusters/3_clusters_Belgium_DC_elec_off_zb_750.json
Check variables of node INLAND : --- 0.000995635986328125 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.0009970664978027344 seconds ---
Check variables of node PCCC_SMR : --- 0.014963626861572266 seconds ---
Check variables of node PCCC_OCGT : --- 0.0 seconds ---
Check variables of node PCCC_CCGT : --- 0.0 seconds ---
Check variables of node PCCC_CHP : --- 0.0009989738464355469 seconds ---
Check variables of node PCCC_WS : --- 0.0 seconds ---
Check variables of node PCCC_BM : --- 0.0 seconds ---
Check variables of node DAC : --- 0.0009965896606445312 seconds ---
Check variables of node H2_STORAGE : --- 0.0009958744049072266 seconds ---
Check variables of node SMR : --- 0.

CPLEX Error  1262: No basis exists.
CPLEX Error  1264: No norms available.


Unable to retrieve  basis  information for variables
Unable to retrieve  dual_norms  information for variables
C:/Users/jocel/Documents/Doctorat/3_clusters/3_clusters_Belgium_DC_elec_off_zb_1000.json


# Distance Off - Coa high renew

In [5]:
name_file = "3_clusters_Belgium_DC.txt"
TimeHorizon = 8760
model = build_model(TimeHorizon,name_file)

Offshore = get_node_in(model, "OFFSHORE")
Zeebrugge = get_node_in(model, "ZEEBRUGGE")
Inland = get_node_in(model, "INLAND")

PV = get_node_in(Inland,"PV")
update_parameter(PV, Parameter("max_capacity", Expression("literal", 100)))
WON = get_node_in(Inland,"WIND_ONSHORE")
update_parameter(WON, Parameter("max_capacity", Expression("literal", 20.5)))
WOFF = get_node_in(Offshore,"WIND_OFFSHORE")
update_parameter(WOFF, Parameter("max_capacity", Expression("literal", 9.3)))
lengths = [40,50,60,70,80,90,100,120,150,200,500,750,1000]
max_power = [1, 0.98, 0.95, 0.91, 0.87, 0.82, 0.74, 0.62, 0.1] +[0] * 4  # consider lost in capacity in HVAC lines for long distance cables
max_HVDC = [100] * 13
sensitivity = 'off_zb'
for length in lengths:
    model.update_global_parameter(Parameter("length_off_zb", Expression("literal", length)))
    name_outfile = file_path + "3_clusters_Belgium_Inf_renew_DC_" + sensitivity + '_' +  str(length) + ".json"
    out = build_and_solve(model, outfile = name_outfile)
    print(name_outfile)
        
        

Check variables of node INLAND : --- 0.000997304916381836 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.0009975433349609375 seconds ---
Check variables of node PCCC_SMR : --- 0.0 seconds ---
Check variables of node PCCC_OCGT : --- 0.0009970664978027344 seconds ---
Check variables of node PCCC_CCGT : --- 0.0 seconds ---
Check variables of node PCCC_CHP : --- 0.0 seconds ---
Check variables of node PCCC_WS : --- 0.000997304916381836 seconds ---
Check variables of node PCCC_BM : --- 0.0 seconds ---
Check variables of node DAC : --- 0.0 seconds ---
Check variables of node H2_STORAGE : --- 0.000997304916381836 seconds ---
Check variables of node SMR : --- 0.0009975433349609375 seconds ---
Check variables of node H2_INTERCONNECTION_NL : --- 0.0 seconds ---
Check variables of node H2_INTERCONNECTION_DE : --- 0.0 seconds ---
Check variables of node H2_INTERCONNECTION : --- 0

CPLEX Error  1262: No basis exists.
CPLEX Error  1264: No norms available.


Unable to retrieve  basis  information for variables
Unable to retrieve  dual_norms  information for variables
C:/Users/jocel/Documents/Doctorat/3_clusters/3_clusters_Belgium_Inf_renew_DC_off_zb_40.json
Check variables of node INLAND : --- 0.000997304916381836 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.000997304916381836 seconds ---
Check variables of node PCCC_SMR : --- 0.0 seconds ---
Check variables of node PCCC_OCGT : --- 0.0009992122650146484 seconds ---
Check variables of node PCCC_CCGT : --- 0.0 seconds ---
Check variables of node PCCC_CHP : --- 0.0 seconds ---
Check variables of node PCCC_WS : --- 0.0009953975677490234 seconds ---
Check variables of node PCCC_BM : --- 0.0 seconds ---
Check variables of node DAC : --- 0.0010075569152832031 seconds ---
Check variables of node H2_STORAGE : --- 0.0 seconds ---
Check variables of node SMR : --- 0.00098633766174

CPLEX Error  1262: No basis exists.
CPLEX Error  1264: No norms available.


Unable to retrieve  basis  information for variables
Unable to retrieve  dual_norms  information for variables
C:/Users/jocel/Documents/Doctorat/3_clusters/3_clusters_Belgium_Inf_renew_DC_off_zb_50.json
Check variables of node INLAND : --- 0.0010101795196533203 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.0009844303131103516 seconds ---
Check variables of node PCCC_SMR : --- 0.0 seconds ---
Check variables of node PCCC_OCGT : --- 0.0 seconds ---
Check variables of node PCCC_CCGT : --- 0.0009975433349609375 seconds ---
Check variables of node PCCC_CHP : --- 0.0 seconds ---
Check variables of node PCCC_WS : --- 0.0 seconds ---
Check variables of node PCCC_BM : --- 0.0009970664978027344 seconds ---
Check variables of node DAC : --- 0.0 seconds ---
Check variables of node H2_STORAGE : --- 0.0009975433349609375 seconds ---
Check variables of node SMR : --- 0.0 seconds --

...

Unable to retrieve  basis  information for variables
Unable to retrieve  dual_norms  information for variables
C:/Users/jocel/Documents/Doctorat/3_clusters/3_clusters_Belgium_Inf_renew_DC_off_zb_500.json
Check variables of node INLAND : --- 0.0009965896606445312 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.000997304916381836 seconds ---
Check variables of node PCCC_SMR : --- 0.0 seconds ---
Check variables of node PCCC_OCGT : --- 0.000997304916381836 seconds ---
Check variables of node PCCC_CCGT : --- 0.0 seconds ---
Check variables of node PCCC_CHP : --- 0.000997304916381836 seconds ---
Check variables of node PCCC_WS : --- 0.0 seconds ---
Check variables of node PCCC_BM : --- 0.0 seconds ---
Check variables of node DAC : --- 0.000997304916381836 seconds ---
Check variables of node H2_STORAGE : --- 0.001995086669921875 seconds ---
Check variables of node SMR : --- 

CPLEX Error  1262: No basis exists.
CPLEX Error  1264: No norms available.


Unable to retrieve  basis  information for variables
Unable to retrieve  dual_norms  information for variables
C:/Users/jocel/Documents/Doctorat/3_clusters/3_clusters_Belgium_Inf_renew_DC_off_zb_750.json
Check variables of node INLAND : --- 0.0009920597076416016 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.000993967056274414 seconds ---
Check variables of node PCCC_SMR : --- 0.0 seconds ---
Check variables of node PCCC_OCGT : --- 0.0010006427764892578 seconds ---
Check variables of node PCCC_CCGT : --- 0.0 seconds ---
Check variables of node PCCC_CHP : --- 0.0 seconds ---
Check variables of node PCCC_WS : --- 0.0009968280792236328 seconds ---
Check variables of node PCCC_BM : --- 0.0 seconds ---
Check variables of node DAC : --- 0.0009970664978027344 seconds ---
Check variables of node H2_STORAGE : --- 0.000997304916381836 seconds ---
Check variables of node SMR : -

CPLEX Error  1262: No basis exists.
CPLEX Error  1264: No norms available.


Unable to retrieve  basis  information for variables
Unable to retrieve  dual_norms  information for variables
C:/Users/jocel/Documents/Doctorat/3_clusters/3_clusters_Belgium_Inf_renew_DC_off_zb_1000.json


# Distance Off - coa high renew elec

In [6]:
name_file = "3_clusters_Belgium_DC.txt"
TimeHorizon = 8760
model = build_model(TimeHorizon,name_file)

Offshore = get_node_in(model, "OFFSHORE")
Zeebrugge = get_node_in(model, "ZEEBRUGGE")
Inland = get_node_in(model, "INLAND")

PV = get_node_in(Inland,"PV")
update_parameter(PV, Parameter("max_capacity", Expression("literal", 100)))
WON = get_node_in(Inland,"WIND_ONSHORE")
update_parameter(WON, Parameter("max_capacity", Expression("literal", 20.5)))
WOFF = get_node_in(Offshore,"WIND_OFFSHORE")
update_parameter(WOFF, Parameter("max_capacity", Expression("literal", 9.3)))
lengths = [40,50,60,70,80,90,100,120,150,200,500,750,1000]
max_power = [1, 0.98, 0.95, 0.91, 0.87, 0.82, 0.74, 0.62, 0.1] +[0] * 4  # consider lost in capacity in HVAC lines for long distance cables
max_HVDC = [100] * 13
sensitivity = 'elec_off_zb'
model.update_global_parameter(Parameter("case_elec", Expression("literal", 1)))
for length in lengths:
    model.update_global_parameter(Parameter("length_off_zb", Expression("literal", length)))
    name_outfile = file_path + "3_clusters_Belgium_Inf_renew_DC_" + sensitivity + '_' +  str(length) + ".json"
    out = build_and_solve(model, outfile = name_outfile)
    print(name_outfile)
        

Check variables of node INLAND : --- 0.0 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.000997304916381836 seconds ---
Check variables of node PCCC_SMR : --- 0.0009975433349609375 seconds ---
Check variables of node PCCC_OCGT : --- 0.0 seconds ---
Check variables of node PCCC_CCGT : --- 0.0 seconds ---
Check variables of node PCCC_CHP : --- 0.0009970664978027344 seconds ---
Check variables of node PCCC_WS : --- 0.0 seconds ---
Check variables of node PCCC_BM : --- 0.0 seconds ---
Check variables of node DAC : --- 0.0009975433349609375 seconds ---
Check variables of node H2_STORAGE : --- 0.000997304916381836 seconds ---
Check variables of node SMR : --- 0.0 seconds ---
Check variables of node H2_INTERCONNECTION_NL : --- 0.0 seconds ---
Check variables of node H2_INTERCONNECTION_DE : --- 0.000997304916381836 seconds ---
Check variables of node H2_INTERCONNECTION : --- 0

CPLEX Error  1262: No basis exists.
CPLEX Error  1264: No norms available.


Unable to retrieve  basis  information for variables
Unable to retrieve  dual_norms  information for variables
C:/Users/jocel/Documents/Doctorat/3_clusters/3_clusters_Belgium_Inf_renew_DC_elec_off_zb_40.json
Check variables of node INLAND : --- 0.0009989738464355469 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.0009968280792236328 seconds ---
Check variables of node PCCC_SMR : --- 0.0 seconds ---
Check variables of node PCCC_OCGT : --- 0.00099945068359375 seconds ---
Check variables of node PCCC_CCGT : --- 0.0 seconds ---
Check variables of node PCCC_CHP : --- 0.0009639263153076172 seconds ---
Check variables of node PCCC_WS : --- 0.0 seconds ---
Check variables of node PCCC_BM : --- 0.0 seconds ---
Check variables of node DAC : --- 0.0009961128234863281 seconds ---
Check variables of node H2_STORAGE : --- 0.0009970664978027344 seconds ---
Check variables of node SMR

CPLEX Error  1262: No basis exists.
CPLEX Error  1264: No norms available.


Unable to retrieve  basis  information for variables
Unable to retrieve  dual_norms  information for variables
C:/Users/jocel/Documents/Doctorat/3_clusters/3_clusters_Belgium_Inf_renew_DC_elec_off_zb_50.json
Check variables of node INLAND : --- 0.000997781753540039 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0009970664978027344 seconds ---
Check variables of node CO2_STORAGE : --- 0.0 seconds ---
Check variables of node PCCC_SMR : --- 0.0019948482513427734 seconds ---
Check variables of node PCCC_OCGT : --- 0.0 seconds ---
Check variables of node PCCC_CCGT : --- 0.0 seconds ---
Check variables of node PCCC_CHP : --- 0.0009970664978027344 seconds ---
Check variables of node PCCC_WS : --- 0.0 seconds ---
Check variables of node PCCC_BM : --- 0.0 seconds ---
Check variables of node DAC : --- 0.000997304916381836 seconds ---
Check variables of node H2_STORAGE : --- 0.0009975433349609375 seconds ---
Check variables of node SMR

...

Unable to retrieve  basis  information for variables
Unable to retrieve  dual_norms  information for variables
C:/Users/jocel/Documents/Doctorat/3_clusters/3_clusters_Belgium_Inf_renew_DC_elec_off_zb_500.json
Check variables of node INLAND : --- 0.000997304916381836 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.000997304916381836 seconds ---
Check variables of node PCCC_SMR : --- 0.0 seconds ---
Check variables of node PCCC_OCGT : --- 0.0009975433349609375 seconds ---
Check variables of node PCCC_CCGT : --- 0.0 seconds ---
Check variables of node PCCC_CHP : --- 0.0 seconds ---
Check variables of node PCCC_WS : --- 0.000997304916381836 seconds ---
Check variables of node PCCC_BM : --- 0.0 seconds ---
Check variables of node DAC : --- 0.0 seconds ---
Check variables of node H2_STORAGE : --- 0.0019943714141845703 seconds ---
Check variables of node SMR : --- 0.014960289

CPLEX Error  1262: No basis exists.
CPLEX Error  1264: No norms available.


Unable to retrieve  basis  information for variables
Unable to retrieve  dual_norms  information for variables
C:/Users/jocel/Documents/Doctorat/3_clusters/3_clusters_Belgium_Inf_renew_DC_elec_off_zb_750.json
Check variables of node INLAND : --- 0.000997304916381836 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0 seconds ---
Check variables of node CO2_STORAGE : --- 0.0009970664978027344 seconds ---
Check variables of node PCCC_SMR : --- 0.0 seconds ---
Check variables of node PCCC_OCGT : --- 0.0009975433349609375 seconds ---
Check variables of node PCCC_CCGT : --- 0.0 seconds ---
Check variables of node PCCC_CHP : --- 0.0 seconds ---
Check variables of node PCCC_WS : --- 0.0009975433349609375 seconds ---
Check variables of node PCCC_BM : --- 0.0 seconds ---
Check variables of node DAC : --- 0.0 seconds ---
Check variables of node H2_STORAGE : --- 0.001994609832763672 seconds ---
Check variables of node SMR : --- 0.0 second

CPLEX Error  1262: No basis exists.
CPLEX Error  1264: No norms available.


Unable to retrieve  basis  information for variables
Unable to retrieve  dual_norms  information for variables
C:/Users/jocel/Documents/Doctorat/3_clusters/3_clusters_Belgium_Inf_renew_DC_elec_off_zb_1000.json
